# Package

In [22]:
import polars as pl
from owid import catalog
import math
import pandas as pd

# Fonction

In [23]:
def type_col(data):
    types_colonnes = data.dtypes
    dictionnaire_types = {col: str(dtype) for col, dtype in zip(data.columns, types_colonnes)}# pour avoir le nom de la colonne qui est de quelle type
    return dictionnaire_types

In [24]:
def categorie_col(data):
    dict_type_col=type_col(data)
    dict_val_col = {}
    for column, dtype in dict_type_col.items():
        if dtype == 'String':
            unique_values = data.select(pl.col(column).unique()).to_series().to_list()#to_serie pour avoir un objet unidimentional et to_list pour pouvoir crée le dictionnaire notame,t pour pouvoir accéder au valeur de la cléf ce qui n'est pas possible avec un obj de tyoe serie
            dict_val_col[column] = unique_values
    return dict_val_col

In [25]:
def table_valeur_manquante(data):
    obs_manquantes = data.select([
    pl.col(obs).is_null().sum()
    for obs in data.columns])
    pct_obs_manquantes = (obs_manquantes / data.height * 100).transpose(include_header=True, header_name="Column")
    return pct_obs_manquantes

# Ember Climate

In [1]:
data = pl.read_csv("yearly_full_release_long_format2.csv",ignore_errors=True,
    truncate_ragged_lines=True,
     separator=',',low_memory=True)

NameError: name 'pl' is not defined

# graph oral

In [2]:
import pandas as pd
import sys
import os
sys.path.append(os.path.join('..', 'Librairies'))
import lib_Owid as lb

In [3]:
data=pd.read_csv("elec_ember_clim.csv")

In [4]:
lb.table_valeur_manquante(data)

Nom de Variable,Nombre de Données Manquantes,Pourcentage de Données Manquantes (%)
str,i64,f64
"""Area""",0,0.0
"""Country code""",20592,5.97
"""Year""",0,0.0
"""Area type""",0,0.0
"""Continent""",20592,5.97
…,…,…
"""Variable""",0,0.0
"""Unit""",0,0.0
"""Value""",19107,5.54


In [5]:
data = data.iloc[:, :-2]

In [6]:
data['Category'].unique()

array(['Capacity', 'Electricity demand', 'Electricity generation',
       'Electricity imports', 'Power sector emissions'], dtype=object)

## Génération d'éléctricité

In [7]:
data['Subcategory'].unique()

array(['Aggregate fuel', 'Fuel', 'Demand', 'Demand per capita', 'Total',
       'Electricity imports', 'CO2 intensity'], dtype=object)

In [8]:
data['Variable'].unique()

array(['Clean', 'Fossil', 'Gas and Other Fossil',
       'Hydro, Bioenergy and Other Renewables', 'Renewables',
       'Wind and Solar', 'Bioenergy', 'Coal', 'Gas', 'Hydro', 'Nuclear',
       'Other Fossil', 'Other Renewables', 'Solar', 'Wind', 'Demand',
       'Demand per capita', 'Total Generation', 'Net Imports',
       'CO2 intensity', 'Total emissions'], dtype=object)

In [9]:
data_gen=data.loc[data['Category'] == "Electricity generation"]

In [10]:
data_gen['Subcategory'].unique()

array(['Aggregate fuel', 'Fuel', 'Total'], dtype=object)

In [11]:
data_gen_test=data_gen.loc[data_gen['Subcategory'] == "Fuel"]

In [12]:
data_gen_test['Variable'].unique()

array(['Bioenergy', 'Coal', 'Gas', 'Hydro', 'Nuclear', 'Other Fossil',
       'Other Renewables', 'Solar', 'Wind'], dtype=object)

In [13]:
def categorize_sc_ernergy(Variable):
    if Variable == 'Bioenergy':
        return 'Energies renouvelables'
    elif Variable == 'Hydro':
        return 'Energies renouvelables'
    elif Variable == 'Other Renewables':
        return 'Energies renouvelables'
    elif Variable == 'Solar':
        return 'Energies renouvelables'
    elif Variable == 'Wind':
        return 'Energies renouvelables'
    elif Variable == 'Coal':
        return 'Energies non renouvelables'
    elif Variable == 'Gas':
        return 'Energies non renouvelables'
    elif Variable == 'Nuclear':
        return 'Energies non renouvelables'
    elif Variable == 'Other Fossil':
        return 'Energies non renouvelables'
    return 'unknown'  # Assurer un retour par défaut

In [14]:
data_gen_test['energy_type'] = data_gen_test['Variable'].apply(categorize_sc_ernergy)

C:\Users\basma\AppData\Local\Temp\ipykernel_4700\3820697324.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_gen_test['energy_type'] = data_gen_test['Variable'].apply(categorize_sc_ernergy)


In [15]:
data_gen_test

,Area,Country code,Year,Area type,Continent,Ember region,EU,OECD,G20,G7,ASEAN,Category,Subcategory,Variable,Unit,Value,energy_type
29,Afghanistan,AFG,2000,Country,Asia,Asia,0.0,0.0,0.0,0.0,0.0,Electricity generation,Fuel,Bioenergy,%,0.00,Energies renouvelables
30,Afghanistan,AFG,2000,Country,Asia,Asia,0.0,0.0,0.0,0.0,0.0,Electricity generation,Fuel,Coal,%,0.00,Energies non renouvelables
31,Afghanistan,AFG,2000,Country,Asia,Asia,0.0,0.0,0.0,0.0,0.0,Electricity generation,Fuel,Gas,%,0.00,Energies non renouvelables
32,Afghanistan,AFG,2000,Country,Asia,Asia,0.0,0.0,0.0,0.0,0.0,Electricity generation,Fuel,Hydro,%,64.58,Energies renouvelables
33,Afghanistan,AFG,2000,Country,Asia,Asia,0.0,0.0,0.0,0.0,0.0,Electricity generation,Fuel,Nuclear,%,0.00,Energies non renouvelables
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
344916,Zimbabwe,ZWE,2022,Country,Africa,Africa,0.0,0.0,0.0,0.0,0.0,Electricity generation,Fuel,Nuclear,TWh,0.00,Energies non renouvelables
344917,Zimbabwe,ZWE,2022,Country,Africa,Africa,0.0,0.0,0.0,0.0,0.0,Electricity generation,Fuel,Other Fossil,TWh,0.00,Energies non renouvelables
344918,Zimbabwe,ZWE,2022,Country,Africa,Africa,0.0,0.0,0.0,0.0,0.0,Electricity generation,Fuel,Other Renewables,TWh,0.00,Energies renouvelables
344919,Zimbabwe,ZWE,2022,Country,Africa,Africa,0.0,0.0,0.0,0.0,0.0,Electricity generation,Fuel,Solar,TWh,0.03,Energies renouvelables


In [16]:
result = data_gen_test.groupby(['Area', 'Year', 'Unit', 'energy_type'])['Value'].sum().reset_index()

In [17]:
result['Area'].unique()

array(['ASEAN', 'Afghanistan', 'Africa', 'Albania', 'Algeria',
       'American Samoa', 'Angola', 'Antigua and Barbuda', 'Argentina',
       'Armenia', 'Aruba', 'Asia', 'Australia', 'Austria', 'Azerbaijan',
       'Bahamas (the)', 'Bahrain', 'Bangladesh', 'Barbados', 'Belarus',
       'Belgium', 'Belize', 'Benin', 'Bermuda', 'Bhutan', 'Bolivia',
       'Bosnia Herzegovina', 'Botswana', 'Brazil', 'Brunei Darussalam',
       'Bulgaria', 'Burkina Faso', 'Burundi', 'Cabo Verde', 'Cambodia',
       'Cameroon', 'Canada', 'Cayman Islands (the)',
       'Central African Republic (the)', 'Chad', 'Chile', 'China',
       'Colombia', 'Comoros (the)',
       'Congo (the Democratic Republic of the)', 'Congo (the)',
       'Cook Islands (the)', 'Costa Rica', "Cote d'Ivoire", 'Croatia',
       'Cuba', 'Cyprus', 'Czechia', 'Denmark', 'Djibouti', 'Dominica',
       'Dominican Republic (the)', 'EU', 'Ecuador', 'Egypt',
       'El Salvador', 'Equatorial Guinea', 'Eritrea', 'Estonia',
       'Eswatini', '

In [18]:
result_f=result.loc[result['Area'] == "Switzerland"]

In [19]:
result_f2=result_f.loc[result_f['Unit'] == "%"]

In [20]:
import plotly.express as px

In [45]:
translation_dict = {
    'Énergie non renouvelable': 'Énergies non renouvelables',
    'Énergie renouvelable' : 'Énergies renouvelables'
}


result_f2['energy_type']=result_f2['energy_type'].replace(translation_dict)


C:\Users\basma\AppData\Local\Temp\ipykernel_4700\64543909.py:7: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [48]:
# Supposons que 'data2' est votre DataFrame contenant les données
fig = px.line(result_f2, 
             x="Year", 
             y="Value",
             labels={"Value": "Part de la production d'électricité (%)", "Year": "Année","energy_type":"Type d'énergies"}, 
             color="energy_type"
            )

fig.update_layout(
    title={
        'text': "Évolution de la part de la production d'électricité à partir de sources d'énergie <br>renouvelable et non renouvelable en Suisse",
        'x': 0.5,  # Centrer le titre
        'xanchor': 'center'
    },
    coloraxis_colorbar={
        'title': "",  
        'title_font_size': 14
    },
    annotations=[
        dict(
            text="Données obtenues via Ember Climate, sous licence CC BY 4.0.",  
            xref="paper", yref="paper",  # Références en coordonnées relatives au papier
            x=0, y=-0.2,  # Positionner en bas au centre
            showarrow=False,
            font=dict(size=12, color="Black"),
            align="center"
        )
    ],
    xaxis_title_standoff=2  # Ajuster l'espace sous l'axe des x pour éviter le chevauchement
)

In [49]:
fig.show()

In [24]:
import plotly.express as px
import plotly.io as pio

In [50]:
fig.write_image("figure_prod_suisse_soutenance.png", width=900, height=500, scale=5)

In [26]:
fig.write_image("figure.svg")

In [ ]:
# Traduction des termes
data_long['Type d'energies'] = data_long['Energy Type'].replace(translation_dict)